In [1]:
! apt install firefox  xvfb > /dev/null
! pip3 install  pyvirtualdisplay selenium webdriver_manager  > /dev/null

In [2]:
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [169]:
#column_names = ["Brand","Model","Ex-showroom","RTO","Insurance","Kits","Total","Speed","CC","Power","Torque","C-ratio","Bore","Stroke","Cylinders","Valves","Cooling","Fuel","capacity","Mileage","Weight","Height","Length","Clearance","Wheelbase","ABS","Rear-brake","Front-brake","Rear-suspension","Front-suspension","Brake-pads","Slipper-clutch","QuickShifter","Warranty","Service-Network"]
#class_names = ["o-dOKno o-bNCMFw o-cJrNdO","o-dOKno o-bNCMFw o-cJrNdO","notavailable","notavailable","notavailable","notavailable","o-Hyyko o-bPYcRG o-eqqVmt","o-cJrNdO","o-cJrNdO"]
common_url = 'https://www.bikewale.com/'
data_class = 'USCqPI'
bike_class = 'o-dOKno o-bNCMFw o-cJrNdO'.replace(" ", ".")
name_class = 'o-zmksK lI0JO_ o-cpnuEd'.replace(" ", ".")
value_class = 'mLrdoz lI0JO_'.replace(" ", ".")
price_class = 'o-Hyyko o-bPYcRG o-eqqVmt'.replace(" ", ".")
#bikessss = ["ktm-bikes/rc-390/","ktm-bikes/duke-390/","yamaha-bikes/r15/","suzuki-bikes/gixxer-sf-250/","ktm-bikes/rc-200/","ktm-bikes/rc-125/"]
collapsed_sections = ["(//div[@class='USCqPI']/div[4]/ul/li[1]/div/div[3])","(//div[@class='USCqPI']/div[4]/ul/li[2]/div/div[3])","(//div[@class='USCqPI']/div[4]/ul/li[3]/div/div[3])","(//div[@class='USCqPI']/div[5]/ul/li/div/div[2])"]
bikes = ["tvs-bikes/apache-rr-310/","bmw-bikes/g-310-r/","zontes-bikes/350r/","ktm-bikes/rc-390/","ktm-bikes/duke-390/","yamaha-bikes/r15/","suzuki-bikes/gixxer-sf-250/","ktm-bikes/rc-200/","ktm-bikes/rc-125/","ktm-bikes/duke-200/","ktm-bikes/duke-125/","royalenfield-bikes/hunter-350/","yamaha-bikes/mt-15/","keeway-bikes/k300-r/","yezdi-bikes/adventure/","yezdi-bikes/roadster/","yezdi-bikes/scrambler/","jawa-bikes/42-bobber/","jawa-bikes/42-fj/","jawa-bikes/perak/","jawa-bikes/350/","jawa-bikes/42/","triumph-bikes/speed-400/","triumph-bikes/scrambler-400/","triumph-bikes/speed-t4/","harleydavidson-bikes/x440/","husqvarna-bikes/vitpilen-250/","husqvarna-bikes/svartpilen-401/","suzuki-bikes/v-strom-sx/","honda-bikes/cb300r/","honda-bikes/cb350rs/","honda-bikes/cb350/","tvs-bikes/apache-rtr-310/"]
results = pd.DataFrame()

In [167]:
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
browser = webdriver.Firefox(options=options)

In [ ]:
def click_collapsed_div(browser,xpath):
  try:
    elem = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
    try:
      browser.implicitly_wait(1)
      elem.click()
      #print(f"Clicked on element: {elem}")
    except Exception as e:
      print(f"Error clicking element: {elem}: {e}")
  except Exception as e:
    print(f"Error finding element")

def get_bike_details_selenium(url):
  print(f"fetching URL: {url}")
  browser.get(url)
  for collapsed_section in collapsed_sections:
    click_collapsed_div(browser,collapsed_section)

  bike_details = {}

  bike_name = browser.find_element(By.CLASS_NAME, bike_class)
  bike_details["Brand"] = bike_name.text.strip().split(" ")[0]
  bike_details["Model"] = bike_name.text.strip()[len(bike_details["Brand"]):]
  bike_price = browser.find_element(By.CLASS_NAME, price_class)
  bike_details["Price"] = bike_price.text.strip()[2:]
  print("Expanded collapsed divs")
  print("Getting bike details")
  element_names = browser.find_elements(By.CLASS_NAME, name_class)
  element_values = browser.find_elements(By.CLASS_NAME, value_class)
  #element_values = browser.find_elements(By.XPATH, f"(//div[@class='{value_class}']/div)")

  for i in range(len(element_names)):
    bike_details[element_names[i].get_attribute("textContent")] = element_values[i].get_attribute("textContent")

  # for i in range(len(column_names)):
  #   try:
  #     element = browser.find_element(By.CLASS_NAME, class_names[i].replace(" ", "."))
  #     if element:
  #       bike_details[column_names[i]] = element.text.strip()
  #     else:
  #       bike_details[column_names[i]] = np.nan
  #   except:
  #     pass

  return bike_details

def find_and_export_bike_data(bikes,results):
  print("Looking for your bikes!")
  for bike in bikes:
    if "bikewale" in bike:
      url = bike
    else:
      url = common_url + bike
    bike_details = get_bike_details_selenium(url)
    bike_data = pd.DataFrame([bike_details]).reset_index(drop=True)
    results = pd.concat([results, bike_data], ignore_index=True)
    print(f"Fetched data for {bike}")

  print("Done! Exporting to csv")
  results.to_csv("bike_data.csv")
  return results


find_and_export_bike_data(bikes,results)

Looking for your bikes!
fetching URL: https://www.bikewale.com/tvs-bikes/apache-rr-310/
Expanded collapsed divs
Getting bike details
Fetched data for tvs-bikes/apache-rr-310/
fetching URL: https://www.bikewale.com/bmw-bikes/g-310-r/
Error clicking element: <selenium.webdriver.remote.webelement.WebElement (session="25f6a143-964c-4ad6-aa5d-ccd9c8dfbbcc", element="6057ba93-5600-4dc9-9b03-9282e61d9a45")>: Message: The element with the reference 6057ba93-5600-4dc9-9b03-9282e61d9a45 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:725:5
getKnownElement@

In [160]:
results.to_csv("bike_data.csv")

In [137]:
browser.get(common_url + bikes[0])
for collapsed_section in collapsed_sections:
    click_collapsed_div(browser,collapsed_section)

Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="8ba9b4b6-67f0-46f7-8324-4ac14228facc")>
Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="88d62935-6edc-471b-804d-3ab77e2ca5e8")>
Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="bf5bf6af-4805-465a-920d-1f9662257546")>
Clicked on element: <selenium.webdriver.remote.webelement.WebElement (session="0cf90c09-5609-4676-a812-6b9dd7b183d4", element="55a353e3-a279-4ec8-a84d-c16418dc1af7")>


In [110]:
element_names = browser.find_elements(By.CLASS_NAME, name_class.replace(" ", "."))
element_values = browser.find_elements(By.CLASS_NAME, value_class.replace(" ", "."))

In [92]:
element_names = browser.find_elements(By.CLASS_NAME, name_class)
element_values = browser.find_elements(By.XPATH, f"(//div[@class='{value_class}'])")

In [111]:
len(element_names)

101

In [112]:
len(element_values)

101

In [70]:
element_values = browser.find_elements(By.CLASS_NAME, value_class)

In [91]:
element_values[38].text.strip()

'33 psi'

In [90]:
element_names[38].text.strip()

'Rear Tyre Pressure (Rider)'

In [147]:
value_class = ".mLrdoz.lI0JO_"
name_class = ".o-zmksK.lI0JO_.o-cpnuEd"
xpath = "(//*[@class='"+value_class+"'])"
res = browser.find_elements(By.CSS_SELECTOR, value_class)
names = browser.find_elements(By.CSS_SELECTOR, name_class)
for i in range(len(res)):
  print(names[i].get_attribute("textContent") +" : "+res[i].get_attribute("textContent"))

Displacement : 155 cc
Max Power : 18.1 bhp @ 10000 rpm 
Max Torque : 14.2 Nm @ 7500 rpm 
Mileage - ARAI : 51.4 kmpl
Mileage - Owner Reported : 45 kmpl
Riding Range : 495 km
Top Speed : 140 kmph
Riding Modes : Track and Street 
Transmission : 6 Speed Manual 
Gear Shifting Pattern : 1 Down 5 Up 
Cylinders : 1 
Bore : 58 mm
Stroke : 58.7 mm
Valves Per Cylinder : 4 
Compression Ratio : 11.6 : 1 
Spark Plugs : 1 Per Cylinder
Cooling System : Liquid Cooled 
Clutch : Assist And Slipper Clutch 
Fuel Tank Capacity : 11 litres
Reserve Fuel Capacity : 1.76 litres
Emission Standard : BS6 Phase 2 
Fuel Type : Petrol 
Front Suspension : Upside Down Front Forks 
Rear Suspension : Linked-Type Monocross Suspension 
Braking System : Dual Channel ABS 
Front Brake Type : Disc 
Front Brake Size : 282 mm
Caliper - Front : 2 Piston
Rear Brake Type : Disc 
Rear Brake Size : 220 mm
Caliper - Rear : 1 Piston
Wheel Type : Alloy 
Front Wheel Size : 17 inch
Rear Wheel Size : 17 inch
Front Tyre Size : 100/80 - 17 


In [128]:
len(res)

0

In [144]:
names[4].get_dom_attribute("class")

'o-zmksK lI0JO_ o-cpnuEd'

In [146]:
# prompt: get text attribute from dom

names[3].get_attribute("textContent")

'Mileage - ARAI'